In [ ]:
# Road_Lane_Detection

import cv2
import numpy as np

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines

def process_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    blur_gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur_gray, 50, 150)
    imshape = frame.shape
    # change the vertices to select the bottom 50% of the frame
    vertices = np.array([[(0,imshape[0]),(0, imshape[0]*0.5), (imshape[1], imshape[0]*0.5), (imshape[1],imshape[0])]], dtype=np.int32)
    edges = region_of_interest(edges, vertices)
    lines = hough_lines(edges, 1, np.pi/180, 20, 20, 100)
    draw_lines(frame, lines)
    return frame


video = cv2.VideoCapture("test6.mp4")
while True:
    ret, frame = video.read()
    if ret:
        frame = process_frame(frame)
        cv2.imshow("Road Lane Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video.release()
cv2.destroyAllWindows()